___

<a href='http://www.pieriandata.com'><img src='../Pierian_Data_Logo.png'/></a>
___
<center><em>Copyright by Pierian Data Inc.</em></center>
<center><em>For more information, visit us at <a href='http://www.pieriandata.com'>www.pieriandata.com</a></em></center>

# Introduction to Cross Validation

In this lecture series we will do a much deeper dive into various methods of cross-validation. As well as a discussion on the general philosphy behind cross validation. A nice official documentation guide can be found here: https://scikit-learn.org/stable/modules/cross_validation.html

## Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Data Example

In [ ]:
df = pd.read_csv("../DATA/Advertising.csv")

In [ ]:
df.head()

----
----
----
## Train | Test Split Procedure 

0. Clean and adjust data as necessary for X and y
1. Split Data in Train/Test for both X and y
2. Fit/Train Scaler on Training X Data
3. Scale X Test Data
4. Create Model
5. Fit/Train Model on X Train Data
6. Evaluate Model on X Test Data (by creating predictions and comparing to Y_test)
7. Adjust Parameters as Necessary and repeat steps 5 and 6

In [ ]:
## CREATE X and y
X = df.drop('sales',axis=1)
y = df['sales']

# TRAIN TEST SPLIT
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# SCALE DATA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

**Create Model**

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
# Poor Alpha Choice on purpose!
model = Ridge(alpha=100)

In [ ]:
model.fit(X_train,y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred

**Evaluation**

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mean_squared_error(y_test,y_pred)

**Adjust Parameters and Re-evaluate**

In [ ]:
model_2 = Ridge(alpha=1)

In [ ]:
model_2.fit(X_train,y_train)

In [ ]:
y_pred = model_2.predict(X_test)

**Another Evaluation**

In [ ]:
mean_squared_error(y_test,y_pred)

Much better! We could repeat this until satisfied with performance metrics. (We previously showed RidgeCV can do this for us, but the purpose of this lecture is to generalize the CV process for any model).

----
----
----
## Train | Validation | Test Split Procedure 

This is often also called a "hold-out" set, since you should not adjust parameters based on the final test set, but instead use it *only* for reporting final expected performance.

0. Clean and adjust data as necessary for X and y
1. Split Data in Train/Validation/Test for both X and y
2. Fit/Train Scaler on Training X Data
3. Scale X Eval Data
4. Create Model
5. Fit/Train Model on X Train Data
6. Evaluate Model on X Evaluation Data (by creating predictions and comparing to Y_eval)
7. Adjust Parameters as Necessary and repeat steps 5 and 6
8. Get final metrics on Test set (not allowed to go back and adjust after this!)

In [ ]:
## CREATE X and y
X = df.drop('sales',axis=1)
y = df['sales']

In [ ]:
######################################################################
#### SPLIT TWICE! Here we create TRAIN | VALIDATION | TEST  #########
####################################################################
from sklearn.model_selection import train_test_split

# 70% of data is training data, set aside other 30%
X_train, X_OTHER, y_train, y_OTHER = train_test_split(X, y, test_size=0.4, random_state=101)
# Remaining 30% is split into evaluation and test sets
# Each is 15% of the original data size
X_eval, X_test, y_eval, y_test = train_test_split(X_OTHER, y_OTHER, test_size=0.5, random_state=101)

In [ ]:
print(f'Original {len(df)} and evaluation {len(X_eval)}' )

In [ ]:
# SCALE DATA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_eval = scaler.transform(X_eval)
X_test = scaler.transform(X_test)

**Create Model**

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
# Poor Alpha Choice on purpose!
model = Ridge(alpha=100)

In [ ]:
model.fit(X_train,y_train)

In [ ]:
y_eval_pred = model.predict(X_eval)

**Evaluation**

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mean_squared_error(y_eval,y_eval_pred)

**Adjust Parameters and Re-evaluate**

In [ ]:
model = Ridge(alpha=1)

In [ ]:
model.fit(X_train,y_train)

In [ ]:
y_eval_pred = model.predict(X_eval)

**Another Evaluation**

In [ ]:
mean_squared_error(y_eval,y_eval_pred)

**Final Evaluation (Can no longer edit parameters after this!)**

In [ ]:
y_final_test_pred = model.predict(X_test)

In [ ]:
mean_squared_error(y_test,y_final_test_pred)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv("../DATA/Advertising.csv")
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


----
----
----
## Cross Validation with cross_val_score

----

<img src="grid_search_cross_validation.png">

----

In [3]:
## CREATE X and y
X = df.drop('sales',axis=1)
y = df['sales']

# TRAIN TEST SPLIT
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# SCALE DATA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [4]:
model = Ridge(alpha=500)

In [5]:
from sklearn.model_selection import cross_val_score

In [6]:
# SCORING OPTIONS:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_val_score(model,X_train,y_train,
                         scoring='neg_mean_squared_error',cv=5)

In [7]:
scores

array([-21.40919138, -14.4453436 , -22.30375678, -18.54557771,
       -15.57730531])

In [8]:
# Average of the MSE scores (we set back to positive)
abs(scores.mean())

18.456234954290476

**Adjust model based on metrics**

In [35]:
model = Ridge(alpha=1)

In [36]:
# SCORING OPTIONS:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_val_score(model2,X_train,y_train,
                         scoring='neg_mean_squared_error',cv=5)

In [37]:
# Average of the MSE scores (we set back to positive)
abs(scores.mean())

3.344839296530695

**Final Evaluation (Can no longer edit parameters after this!)**

In [38]:
# Need to fit the model first!
model.fit(X_train,y_train)

Ridge(alpha=1)

In [39]:
y_final_test_pred = model.predict(X_test)

In [44]:
print(f'This is reported to the boss \n the mean squared error is {mean_squared_error(y_test,y_final_test_pred)}')


This is reported to the boss 
 the mean square error is 2.3190215794287514


----
----
----

# Cross Validation with cross_validate

The cross_validate function differs from cross_val_score in two ways:

It allows specifying multiple metrics for evaluation.

It returns a dict containing fit-times, score-times (and optionally training scores as well as fitted estimators) in addition to the test score.

For single metric evaluation, where the scoring parameter is a string, callable or None, the keys will be:
        
        - ['test_score', 'fit_time', 'score_time']

And for multiple metric evaluation, the return value is a dict with the following keys:

    ['test_<scorer1_name>', 'test_<scorer2_name>', 'test_<scorer...>', 'fit_time', 'score_time']

return_train_score is set to False by default to save computation time. To evaluate the scores on the training set as well you need to be set to True.

In [45]:
## CREATE X and y
X = df.drop('sales',axis=1)
y = df['sales']

# TRAIN TEST SPLIT
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# SCALE DATA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [46]:
model = Ridge(alpha=100)

In [47]:
from sklearn.model_selection import cross_validate

In [51]:
# SCORING OPTIONS:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_validate(model,X_train,y_train,
                         scoring=['neg_mean_absolute_error','neg_mean_squared_error','max_error','r2'],cv=50)

In [52]:
scores

{'fit_time': array([0.        , 0.00099993, 0.00103354, 0.        , 0.        ,
        0.00100064, 0.00100136, 0.00099897, 0.        , 0.0010004 ,
        0.        , 0.00099945, 0.        , 0.00099993, 0.        ,
        0.00100207, 0.00100088, 0.        , 0.00096822, 0.        ,
        0.        , 0.        , 0.        , 0.00100017, 0.0009985 ,
        0.        , 0.00100303, 0.00099993, 0.        , 0.00098968,
        0.00099993, 0.        , 0.        , 0.00099969, 0.        ,
        0.        , 0.00099969, 0.0010016 , 0.00096965, 0.        ,
        0.00099945, 0.        , 0.00097322, 0.        , 0.00100255,
        0.        , 0.        , 0.00099683, 0.00099301, 0.        ]),
 'score_time': array([0.00100899, 0.        , 0.        , 0.0010004 , 0.00099969,
        0.00100064, 0.00099921, 0.        , 0.00100112, 0.00099945,
        0.00099921, 0.00099993, 0.00100017, 0.        , 0.00099993,
        0.00099969, 0.        , 0.00100183, 0.        , 0.00097036,
        0.00096798, 

In [53]:
pd.DataFrame(scores)

,fit_time,score_time,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_max_error,test_r2
0,0.000000,0.001009,-2.212863,-11.317185,-5.768734,0.780826
1,0.001000,0.000000,-3.102003,-9.726685,-3.360340,0.565557
2,0.001034,0.000000,-0.464445,-0.332716,-0.913622,-0.077138
3,0.000000,0.001000,-2.144795,-5.622735,-3.365334,0.543856
4,0.000000,0.001000,-0.632758,-0.537568,-1.125997,0.940108
5,0.001001,0.001001,-3.775118,-18.932034,-5.894107,0.292231
6,0.001001,0.000999,-2.511890,-11.696155,-5.784770,0.421809
7,0.000999,0.000000,-2.106696,-8.082231,-4.767853,0.761461
8,0.000000,0.001001,-2.100443,-6.387890,-3.337340,0.809241
9,0.001000,0.000999,-1.664000,-4.397743,-3.345134,0.706861


In [54]:
pd.DataFrame(scores).mean()

fit_time                        0.000519
score_time                      0.000741
test_neg_mean_absolute_error   -2.006895
test_neg_mean_squared_error    -7.145719
test_max_error                 -3.406394
test_r2                        -2.347470
dtype: float64

**Adjust model based on metrics**

In [82]:
model = Ridge(alpha=0.5)

In [83]:
# SCORING OPTIONS:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_validate(model,X_train,y_train,
                         scoring=['neg_mean_absolute_error','neg_mean_squared_error','max_error','r2'],cv=5)

In [84]:
pd.DataFrame(scores).mean()

fit_time                        0.000998
score_time                      0.000798
test_neg_mean_absolute_error   -1.319874
test_neg_mean_squared_error    -3.343919
test_max_error                 -5.156009
test_r2                         0.862784
dtype: float64

**Final Evaluation (Can no longer edit parameters after this!)**

In [85]:
# Need to fit the model first!
model.fit(X_train,y_train)

Ridge(alpha=0.5)

In [86]:
y_final_test_pred = model.predict(X_test)

In [87]:
mean_squared_error(y_test,y_final_test_pred)

2.3086084522858417

----
----